# 04 - Client MCP data.gouv.fr

**Objectif** : Se connecter au serveur MCP pour récupérer les données fraîches

**Serveur** : https://mcp.data.gouv.fr/mcp

**Protocole** : Streamable HTTP + JSON-RPC 2.0

**Tools disponibles** :
- `search_datasets` : Rechercher des datasets
- `get_dataset_info` : Infos détaillées d'un dataset
- `list_dataset_resources` : Lister les ressources d'un dataset
- `query_resource_data` : Interroger les données d'une ressource

## 1. Configuration

In [ ]:
import os
import json
import httpx
from dotenv import load_dotenv

load_dotenv("../.env")

MCP_URL = os.getenv("MCP_DATAGOUV_URL", "https://mcp.data.gouv.fr/mcp")

print(f"✅ MCP URL : {MCP_URL}")

## 2. Client MCP basique

In [ ]:
class MCPClient:
    """
    Client simple pour le serveur MCP data.gouv.fr.
    Utilise JSON-RPC 2.0 sur Streamable HTTP.
    """

    def __init__(self, base_url: str):
        self.base_url = base_url
        self.session_id = None
        self._request_id = 0

    def _next_id(self) -> int:
        self._request_id += 1
        return self._request_id

    def _call(self, method: str, params: dict = None) -> dict:
        """
        Appel JSON-RPC au serveur MCP.
        """
        payload = {
            "jsonrpc": "2.0",
            "id": self._next_id(),
            "method": method,
            "params": params or {}
        }

        # Headers requis pour le protocole MCP Streamable HTTP
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json, text/event-stream"
        }
        if self.session_id:
            headers["Mcp-Session-Id"] = self.session_id

        # Forcer HTTP/1.1 pour éviter les problèmes avec HTTP/2
        with httpx.Client(timeout=60, http2=False) as client:
            response = client.post(self.base_url, json=payload, headers=headers)

            # Récupérer le session ID si présent
            if "mcp-session-id" in response.headers:
                self.session_id = response.headers["mcp-session-id"]

            response.raise_for_status()

        # Gérer les réponses SSE (text/event-stream)
        content_type = response.headers.get("content-type", "")
        if "text/event-stream" in content_type:
            # Parser les événements SSE pour extraire le JSON
            for line in response.text.split("\n"):
                if line.startswith("data:"):
                    data_str = line[5:].strip()
                    if data_str:
                        parsed = json.loads(data_str)
                        if "error" in parsed:
                            raise Exception(f"MCP Error: {parsed['error']}")
                        return parsed.get("result", {})
            return {}

        result = response.json()

        if "error" in result:
            raise Exception(f"MCP Error: {result['error']}")

        return result.get("result", {})

    def initialize(self) -> dict:
        """Initialiser la session MCP."""
        return self._call("initialize", {
            "protocolVersion": "2024-11-05",
            "capabilities": {},
            "clientInfo": {
                "name": "poc-datagouv",
                "version": "0.1.0"
            }
        })

    def list_tools(self) -> list:
        """Lister les tools disponibles."""
        result = self._call("tools/list")
        return result.get("tools", [])

    def call_tool(self, name: str, arguments: dict = None) -> dict:
        """Appeler un tool MCP."""
        return self._call("tools/call", {
            "name": name,
            "arguments": arguments or {}
        })

# Créer le client
mcp = MCPClient(MCP_URL)
print("✅ Client MCP créé")

## 3. Initialisation de la session

In [ ]:
# Initialiser la connexion
init_result = mcp.initialize()

print(f"✅ Session initialisée")
print(f"   Session ID : {mcp.session_id}")
print(f"   Server info : {init_result.get('serverInfo', {})}")

In [ ]:
# Lister les tools disponibles
tools = mcp.list_tools()

print(f"🔧 {len(tools)} tools disponibles :\n")
for tool in tools:
    print(f"• {tool['name']}")
    print(f"  {tool.get('description', 'Pas de description')[:80]}")
    print()

## 4. Test des tools

In [ ]:
# Test : search_datasets
result = mcp.call_tool("search_datasets", {
    "query": "qualité air",
    "page_size": 3
})

print("🔍 Recherche 'qualité air' :\n")

# Le serveur retourne du texte formaté, pas du JSON
if result.get("content"):
    content = result["content"][0]
    if content.get("type") == "text":
        print(content["text"])

In [ ]:
# Test : get_dataset_info avec un ID connu
DATASET_ID = "668889232ab126cfc336b4fd"  # PLN 2022 Qualité AIR

result = mcp.call_tool("get_dataset_info", {
    "dataset_id": DATASET_ID
})

print(f"📄 Info dataset :\n")

if result.get("content"):
    content = result["content"][0]
    if content.get("type") == "text":
        print(content["text"])

In [ ]:
# Test : list_dataset_resources
result = mcp.call_tool("list_dataset_resources", {
    "dataset_id": DATASET_ID
})

print(f"📦 Ressources du dataset :\n")

if result.get("content"):
    content = result["content"][0]
    if content.get("type") == "text":
        print(content["text"])

## 5. Fonctions utilitaires

In [ ]:
import re

def extract_text(result: dict) -> str:
    """Extrait le texte d'une réponse MCP."""
    if result.get("content"):
        content = result["content"][0]
        if content.get("type") == "text":
            return content["text"]
    return ""


def parse_search_results(text: str) -> list[dict]:
    """
    Parse le texte de recherche en liste de datasets.
    Format attendu :
    1. Titre
       ID: xxx
       Organization: xxx
       ...
    """
    datasets = []
    # Pattern pour chaque dataset
    pattern = r'(\d+)\.\s+(.+?)\n\s+ID:\s+(\S+)\n\s+Organization:\s+(.+?)\n'
    
    for match in re.finditer(pattern, text):
        datasets.append({
            "rank": int(match.group(1)),
            "title": match.group(2).strip(),
            "id": match.group(3).strip(),
            "organization": match.group(4).strip()
        })
    
    return datasets


def search_datasets_mcp(query: str, page_size: int = 5) -> list[dict]:
    """
    Recherche de datasets via MCP.
    Retourne une liste de dicts avec les infos essentielles.
    """
    result = mcp.call_tool("search_datasets", {
        "query": query,
        "page_size": page_size
    })
    
    text = extract_text(result)
    return parse_search_results(text)


def get_dataset_info_mcp(dataset_id: str) -> str:
    """
    Récupère les infos complètes d'un dataset.
    Retourne le texte formaté.
    """
    result = mcp.call_tool("get_dataset_info", {
        "dataset_id": dataset_id
    })
    return extract_text(result)


def list_resources_mcp(dataset_id: str) -> str:
    """
    Liste les ressources d'un dataset.
    Retourne le texte formaté.
    """
    result = mcp.call_tool("list_dataset_resources", {
        "dataset_id": dataset_id
    })
    return extract_text(result)


def query_resource_mcp(resource_id: str, sql_query: str = None) -> str:
    """
    Interroge les données d'une ressource.
    Retourne le texte formaté.
    """
    params = {"resource_id": resource_id}
    if sql_query:
        params["query"] = sql_query
    
    result = mcp.call_tool("query_resource_data", params)
    return extract_text(result)


print("✅ Fonctions utilitaires définies")

## 6. Exemple complet : de la recherche aux données

In [ ]:
# Scénario : trouver des données sur les bornes de recharge électrique

# 1. Recherche
datasets = search_datasets_mcp("bornes recharge électrique", page_size=3)
print("🔍 Datasets trouvés :\n")
for ds in datasets:
    print(f"• {ds['title']}")
    print(f"  ID: {ds['id']}")
    print(f"  Org: {ds['organization']}")
    print()

In [ ]:
# 2. Prendre le premier dataset et lister ses ressources
if datasets:
    dataset_id = datasets[0]["id"]
    print(f"📦 Ressources de '{datasets[0]['title']}' :\n")
    
    resources_text = list_resources_mcp(dataset_id)
    print(resources_text)

In [ ]:
# 3. Voir les infos détaillées du dataset
if datasets:
    print(f"📄 Infos détaillées :\n")
    info_text = get_dataset_info_mcp(datasets[0]["id"])
    print(info_text[:1500])  # Limiter l'affichage

## 7. Résumé

**Client MCP implémenté** avec :
- `MCPClient` : Classe pour gérer la session et les appels JSON-RPC
- `search_datasets_mcp(query)` : Recherche textuelle
- `get_dataset_info_mcp(id)` : Métadonnées complètes
- `list_resources_mcp(id)` : Liste des fichiers
- `query_resource_mcp(id, sql)` : Interrogation SQL

---

## Prochaine étape

**Notebook 05** : Orchestration LLM
- Combiner recherche vectorielle Mediatech + données fraîches MCP
- Générer une réponse en langage naturel avec Albert